<center>Национальный исследовательский университет ИТМО<br/>Факультет информационных технологий и программирования<br/>Прикладная математика и информатика</center>

## <center>Математическая статистика</center>
### <center>Отчёт по лабораторной работе №4</center>

<div style="text-align: right"><b>Работу выполнили:</b><br/>Белоцерковченко Андрей M3337<br/>Смирнов Андрей M3337<br/</div>

<center>Санкт-Петербург<br/>2023</center>

# Вариант 2

## Задача 1

In [2]:
import numpy as np
from scipy.special import comb
from scipy.stats import norm
from scipy.stats import rankdata
import pandas as pd
import math
df = pd.read_csv('exams_dataset.csv')
alpha = 0.05

### Гипотезы

$H_0:$ статистически количество мужчин и женщин неразличимо 

$H_1:$ статистически их количество различается

Будем использовать критерий $X ^ 2$ для проверки на независимость


Критерий независимости хи-квадрат (также известный как тест ассоциации хи-квадрат) позволяет определить, связаны ли две категориальные переменные. Если две переменные связаны, вероятность того, что одна переменная имеет определённое значение, зависит от значения другой переменной.

Вычисления критерия независимости хи-квадрат основаны на наблюдаемых частотах, которые представляют собой количество наблюдений в каждой объединённой группе. Тест сравнивает наблюдаемые частоты с частотами, которые можно было бы ожидать, если бы две переменные не были связаны.

In [8]:
from scipy.stats import chisquare

def calc_by_field(field: str):
    # Создание таблицы сопряженности
    contingency_table = pd.crosstab(df[field], columns='count')
    
    # Применение критерия хи-квадрат
    res = chisquare(contingency_table['count'])
    print(f'Статистика критерия хи-квадрат: {res.statistic}')
    print(f'P-значение: {res.pvalue}')
    print(contingency_table)

In [9]:
calc_by_field('gender')

Статистика критерия хи-квадрат: 0.144
P-значение: 0.7043364134884518
col_0   count
gender       
female    494
male      506


Пояснение:

`statistic`: Это значение тестовой статистики, которое сравнивается с распределением хи-квадрат.
`pvalue`: Это p-значение теста. Он используется для оценки статистической значимости. Если p-значение меньше уровня значимости (обычно 0.05), то нулевая гипотеза о том, что наблюдаемое распределение соответствует ожидаемому, может быть отвергнута.

И сама наша статистика хи квадрат. На основе P-значения можем сделать вывод о том, что гипотеза H0 применима

Таким образом результаты теста:
- Используется статистика $X^2$
- Степени свободы при распределении хи квадрат - `(rows - 1) * (cols - 1)`
- Критическая область двусторонняя

Статистика: Это значение тестовой статистики (в данном случае, статистика Хи-квадрат). Это число используется для сравнения с критической областью и принятия решения о том, следует ли отвергнуть нулевую гипотезу.

Распределение при условии нулевой гипотезы: Это распределение, которое принимается во внимание при проверке нулевой гипотезы. В данном случае, оно описывается как распределение Хи-квадрат с 

Тип критической области: Это указание на тип критической области. В данном случае, говорится о двусторонней критической области. Двусторонний тест означает, что мы исследуем оба конца распределения (хвосты), и отвергаем нулевую гипотезу, если статистика попадает в один из этих хвостов.

### То же самое для этнической принадлежности

In [10]:
calc_by_field("race/ethnicity")

Статистика критерия хи-квадрат: 192.98999999999998
P-значение: 1.2070807175456277e-40
col_0           count
race/ethnicity       
group A            77
group B           204
group C           324
group D           261
group E           134


Здесь у нас получается что $p << \alpha$ при $\alpha = 0.05$, а следовательно гипотеза $H0$ должна быть отвергнута

Таким образом результаты теста:
- Используется статистика $X^2$
- Степени свободы при распределении хи квадрат - `(rows - 1) * (cols - 1)`
- Критическая область двусторонняя

## Задача 2
В задаче мы хотим проверить, что выборки результатов по математике и письменной части имеют одинаковое распределение. Более формально: $$H_0: F_X = F_Y$$ $$H_1: F_X \neq F_Y $$ Здесь и далее уровень значимости $\alpha = 0.05$

### Критерий знаков
Пусть результаты по математике будут равны $X_1, X_2, X_3 ... X_n$, а по письменной части $Y_1, Y_2, Y_3 ... Y_n$. Возьмём последовательность $Z$, такую что $sign(X_1 - Y_1), sign(X_2 - Y_2), sign(X_3 - Y_3) ... sign(X_n - Y_n)$. Выкинем из рассмотрения все нули (случаи, когда $X_i = Y_i$). Мы хотим проверить, правда ли, что $P(Z > 0) = 0.5$.
Переформулируем гипотезы: $$H_0: p = \cfrac{1}{2}$$ $$H_1: p \neq \cfrac{1}{2}$$
Пусть $w$ - количество $Z_i = 1$. Если $H_0$ выполняется, то $w \sim B(n, 0.5)$ А дальше просто проверяем, что $b = \cfrac{\sum\limits_{i = 0}^w C_n^i}{2^n}$ попадает в критическую область $(\cfrac{\alpha}{2}, 1 - \cfrac{\alpha}{2})$, где $\alpha$ - уровень значимости равный $0.05$

In [111]:
sign = lambda x: math.copysign(1, x)
test = (df['math score'] - df['writing score']).apply(sign)
w = test.gt(0).sum()
cSum = 0
for i in range(0, w + 1):
    cSum += comb(1000, i, exact=True)

b = cSum / (2 ** 1000)
print("Sign stat:", b)
print("Check if stat in critical region:", (alpha / 2) < b < 1 - (alpha / 2))

Sign stat: 0.0024308681959370596
Check if stat in critical region: False


## Критерий Ван дер Вардена

Определим величину $A_{ij}$: $$A_{ij} = \Phi^{-1}(\cfrac{R(X_{ij})}{N + 1})$$ где $R(X_{ij})$ - ранг $i$-ого элемента $j$-ой выборки в объединённой выборке, N - размер объединённой выборки. Также введём величины $\overline{A_j}$ и $s^2$: $$\overline{A_j} = \cfrac{1}{n_j}\sum\limits_{i = 1}^{n_j}A_{ij}$$ $$s^2 = \cfrac{1}{N - 1}\sum\limits_{j = 1}^k\sum\limits_{i = 1}^{n_j}A_{ij}^2$$
Тогда статистика считается по формуле $$T = \cfrac{1}{s^2}\sum\limits_{j = 1}n_j\overline{A_j}^2$$
Поскольку тест Ван дер Вардена описан для $k$ выборок - в оригинале гипотеза $H_1$ звучит так: "хотя бы одна из выборок имеет тенденцию давать более крупные значения, чем другие" Но поскольку в нашем случае мы проверяем однородность только двух выборок - можем оставить $H_1$ без изменения.

При этом нуль-гипотеза отвергается, если статистика попадёт в промежуток $T > \chi^2_{\alpha, k - 1}$

In [112]:
def count_mean_rank(f, s, d):
    n = len(f)
    m = len(s)
    k = n + m + 1
    R = 0
    for x in f:
        R += norm.ppf(d[x] / k)
        
    return R / n

def count_variance(f, s):
    n = len(f)
    m = len(s)
    
    R = 0
    for x in f:
        R += x
    for y in s:
        R += y
        
    return R / (n + m - 1)

a = df['math score']
b = df['writing score']
u = np.concatenate((a, b))
number_to_rank = dict(zip(u, rankdata(u)))
stat = ((count_mean_rank(a, b, number_to_rank) ** 2) * len(a) + 
        (count_mean_rank(b, a, number_to_rank) ** 2) * len(b)) / count_variance(a, b)
print("Van der Waerden stat:", stat)
print("Check if stat in critical region:", chi2.ppf(alpha, 1) < stat)

Van der Waerden stat: 0.10906549996717955
Check if stat in critical region: True


Таким образом, оба критерия показали неоднородность результатов экзамена по математике и письменной части.  

## Задача 3
Верно ли что прошедшие подготовительные курсы лучше сдали экзамены?

$H_0:$ результаты экзаменов не зависят от прохождения подготовительных курсов

$H_1:$ прошедшие курсы сдали экзамены лучше

Выполним t-тест с выборками по тем кто прошел курс и тем кто этого не сдалал

In [12]:
from scipy.stats import ttest_ind

had_course = df[df['test preparation course'] == 'completed']['math score']
no_course = df[df['test preparation course'] == 'none']['math score']
ans = ttest_ind(had_course, no_course)

In [14]:
print(f'Статистика: {ans.statistic}')
print(f'p значение = {ans.pvalue}')
print(ans.df)

Статистика: 5.5589194118951095
p значение = 3.483493224877811e-08
998.0


Значение p очень мало, а значит эту гипотезу можно отвергать

### Метод Манна Уитни

In [15]:
from scipy.stats import mannwhitneyu

ans = mannwhitneyu(had_course, no_course)

print(f'Статистика: {ans.statistic}')
print(f'p значение = {ans.pvalue}')

Статистика: 140075.0
p значение = 2.2533705495929116e-07


Также значение очень мало и значит гипотезу можно отвергать